In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary


In [2]:
# Data transformations for CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224,224)), # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])

In [3]:
# Loading CIFAR-10 datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:14<00:00, 12.1MB/s]


In [4]:
batch_size = 128
num_classes = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
model = models.resnet18(pretrained=True)
# print(model.parameters())
# Freeze the early layers (feature extractor)
for param in model.parameters():
  # print(param)
  param.requires_grad = False

# Replace the classifier layer (fully connected layer)
num_ftrs = model.fc.in_features
print(num_ftrs)
model.fc = nn.Linear(num_ftrs, num_classes)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 207MB/s]

512


In [6]:
# move Model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

summary(model, (3, 224,224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [12]:

# Optimizer and loss function
optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

# Training function
def train_model(model, train_loader, test_loader, epochs):
    model.train()
    for epoch in range(epochs):
        total_running_loss = 0
        correct_train = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Loss and accuracy calculation
            total_running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            correct_train += preds.eq(labels).sum().item()

        # Average loss and accuracy for the training set
        avg_loss = total_running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / len(train_loader.dataset)
        train_losses.append(avg_loss)
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        running_val_loss = 0
        correct_val = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # Loss and accuracy calculation
                running_val_loss += loss.item() * inputs.size(0)
                preds = outputs.argmax(dim=1)
                correct_val += preds.eq(labels).sum().item()

        # Average loss and accuracy for the validation set
        val_loss = running_val_loss / len(test_loader.dataset)
        val_accuracy = correct_val / len(test_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)

        # Print epoch summary
        print(f"Epoch [{epoch + 1}/{epochs}]")
        print(f"Training    - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy * 100:.2f}%")
        print(f"Validation  - Loss: {val_loss:.4f}, Accuracy: {val_accuracy * 100:.2f}%\n")

# Train and evaluate the model
train_model(model, train_loader, test_loader, epochs)



Epoch [1/5]
Training    - Loss: 0.5332, Accuracy: 81.31%
Validation  - Loss: 0.5683, Accuracy: 80.45%

Epoch [2/5]
Training    - Loss: 0.5128, Accuracy: 82.30%
Validation  - Loss: 0.5637, Accuracy: 80.86%

Epoch [3/5]
Training    - Loss: 0.5095, Accuracy: 82.25%
Validation  - Loss: 0.5620, Accuracy: 80.94%

Epoch [4/5]
Training    - Loss: 0.5074, Accuracy: 82.38%
Validation  - Loss: 0.5709, Accuracy: 80.31%

Epoch [5/5]
Training    - Loss: 0.5066, Accuracy: 82.51%
Validation  - Loss: 0.5793, Accuracy: 80.41%



In [23]:
# Evaluate function
def evaluate_model(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input, label in test_loader:
      input, label = input.to(device), label.to(device)
      outputs = model(input)
      loss = criterion(outputs, label)
      test_loss += loss.item() * input.size(0)
      pred = outputs.argmax(dim=1)
      correct += pred.eq(label).sum().item()
  avg_loss = test_loss / len(test_loader.dataset)
  accuracy = correct/  len(test_loader.dataset)
  print(f'Test Loss : {avg_loss: .4f}, Test Accuracy : {accuracy:.4f}')

evaluate_model(model, test_loader)

Test Loss :  0.4823, Test Accuracy : 0.9021


In [10]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary


# Data transformations for CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224,224)), # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229, 0.224, 0.225])  # Normalization for pre-trained models
])


# Loading CIFAR-10 datasets
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

batch_size = 128
num_classes = 10
epochs = 5
learning_rate = 0.001
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


model = models.resnet18(pretrained=True)
# print(model)

# Freeze the early layers (feature extractor)
for param in model.parameters():
  # print(param)
  param.requires_grad = False

# unfreeze the last two layers (layer4)
for param in model.layer4.parameters():
  param.requires_grad = True


# Replace the classifier layer (fully connected layer)
num_ftrs = model.fc.in_features
print(num_ftrs)
model.fc = nn.Linear(num_ftrs, num_classes)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Print model summary
summary(model, (3, 224, 224))

512
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1,

In [22]:
# Optimizer and Loss function
# Include parameters of both the classifier and unfrozen layers
optimizer = optim.Adam([
    {'params': model.layer4.parameters()},
    {'params': model.fc.parameters()}
], lr=learning_rate)
criterion = nn.CrossEntropyLoss()

def train_model (model, train_loader, epochs):
  model.train()
  for epoch in range(epochs):
    total_running_loss = 0
    correct = 0
    for input, label in train_loader:
      input,label = input.to(device), label.to(device)
      optimizer.zero_grad()
      outputs = model(input)
      loss = criterion(outputs,label)
      loss.backward()
      optimizer.step()
      total_running_loss += loss.item()  * input.size(0)
      pred = outputs.argmax(dim=1)
      correct += pred.eq(label).sum().item()

    avg_loss = total_running_loss / len(train_loader.dataset)
    accuracy = correct / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1} / {epochs}], Training Loss : {avg_loss:.4f}, Accuracy{accuracy:.4f}')


train_model(model, train_loader, epochs)


Epoch [1 / 5], Training Loss : 0.0561, Accuracy0.9811
Epoch [2 / 5], Training Loss : 0.0351, Accuracy0.9883
Epoch [3 / 5], Training Loss : 0.0312, Accuracy0.9897
Epoch [4 / 5], Training Loss : 0.0231, Accuracy0.9920
Epoch [5 / 5], Training Loss : 0.0290, Accuracy0.9900


In [24]:
def evaluate_model(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for input,label in test_loader:
      input, label = input.to(device), label.to(device)
      output = model(input)
      loss = criterion(output, label)
      test_loss += loss.item() * input.size(0)
      pred = output.argmax(dim=1)
      correct += pred.eq(label).sum().item()

  avg_loss = test_loss / len(test_loader.dataset)
  accuracy = correct / len(test_loader.dataset)
  print(f"Test Loss: {avg_loss*100:.2f}, Test Accuracy: {accuracy*100:.2f}")

evaluate_model(model, test_loader)



Test Loss: 48.23, Test Accuracy: 90.21
